## Imports

In [49]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import os
from dataclasses import dataclass
from tabulate import tabulate
import csv

## Definindo Classe Comentário


In [50]:
@dataclass
class Comentario:
    autor: str      # Nome do autor do comentário
    nota: float     # Nota atribuída pelo espectador
    conteudo: str   # Texto do comentário
    criticas: int = None  # Número de críticas feitas pelo autor
    seguidores: int = None  # Número de seguidores do autor

## Definindo Classe Filme com Comentários

In [51]:
@dataclass
class Movie:
    name: str    # Nome do filme
    grade: float # Nota do filme
    resume: str  # Resumo da crítica do filme
    comentarios: list[Comentario] = None

## Pegando os filmes das 4 primeiras páginas

In [52]:
# URL base do site AdoroCinema
URL = "https://www.adorocinema.com/filmes/melhores/adorocinema/?page="

# Lista para armazenar os filmes
MOVIES_DETAILS = []

# Loop para percorrer as primeiras 4 páginas
for i in range(1, 3):
    # Requisição HTTP para a página atual
    response = requests.get(URL + str(i))
    content = response.content

    # Criando objeto BeautifulSoup para analisar o HTML
    soup = BeautifulSoup(content, 'html.parser')

    # Encontrando todos os links de filmes
    links = soup.find_all('a', class_='meta-title-link')

    # Processando cada link de filme
    for link in links:
        # Acessando a página de críticas do filme
        response = requests.get('https://www.adorocinema.com' + link.get('href') + 'criticas-adorocinema/')
        content = response.content

        # Analisando o HTML da página de críticas
        soup = BeautifulSoup(content, 'html.parser')

        # Extraindo o texto da crítica
        details = soup.find_all('p', class_='bo-p')
        resumo = ''
        for detail in details:
            resumo += detail.getText()

        # Extraindo o título do filme
        title = soup.find('div', class_='title').getText()

        # Extraindo a nota do filme
        grade = soup.find('span', class_='note').getText()

        # Adicionando o filme à lista, tratando os dados extraídos
        MOVIES_DETAILS.append(Movie(
            re.sub('\\s\\s+', '', title.replace('\n', '')),  # Limpa o título
            float(grade.replace(',', '.')),  # Converte a nota para float
            resumo,  # Resumo da crítica
            []  # Lista vazia para comentários
        ))

    print(f"Página {i} processada. Total de filmes até agora: {len(MOVIES_DETAILS)}")

# Exibindo os filmes coletados em formato de tabela
print(tabulate([[movie.name, movie.grade, movie.resume[:100] + "..."]
                for movie in MOVIES_DETAILS],
               headers=["Nome", "Nota", "Resumo"]))

Página 1 processada. Total de filmes até agora: 15
Página 2 processada. Total de filmes até agora: 30
Nome                                               Nota  Resumo
-----------------------------------------------  ------  -------------------------------------------------------------------------------------------------------
Medianeras: Buenos Aires na Era do Amor Virtual       5  Sabe aqueles filmes que começam meio estranho e de repente mudam, capturando sua atenção minuto a mi...
Wall-E                                                5  É sempre bom assistir a filmes da era do cinema mudo. Trata-se de outro estilo, refletido não só na ...
Hotel Transilvânia                                    5  Os filmes de animação têm uma missão complicada: agradar baixinhos e altinhos. Durante anos, a Disne...
A Era do Gelo 3                                       5  Quem conhece Manny, Ellie, Sid, Diego, Crash e Eddie e, claro, Scrat, já tem uma ideia do que o agua...
Megamente                    

# Extraindo Comentários do Filme

In [53]:
def extrair_comentarios(url_filme, max_comentarios=40):

    comentarios = []

    # URL para a seção de críticas de espectadores deste filme
    url = url_filme + "/criticas/espectadores/"

    # Realiza uma requisição HTTP para a página de críticas
    response = requests.get(url)
    if response.status_code != 200:
        print(f"Erro ao acessar a página: {url}")
        return comentarios

    content = response.content

    # Cria um objeto BeautifulSoup para analisar o HTML da página
    soup = BeautifulSoup(content, 'html.parser')

    # Busca o elemento de paginação para determinar quantas páginas de comentários existem
    pagination_div = soup.find('div', class_='pagination-item-holder')

    # Se não encontrar paginação, assume que há apenas uma página
    if pagination_div is None:
        last_page = 1
    else:
        pagination_items = pagination_div.find_all(class_='item')
        if len(pagination_items) == 0:
            last_page = 1
        else:
            # Pega o número da última página de comentários
            try:
                last_page = int(pagination_items[-1].text)
            except (ValueError, IndexError):
                last_page = 1

    # Variável para controlar o número de comentários extraídos
    comentarios_extraidos = 0

    # Percorre as páginas de comentários até atingir o limite
    for i in range(1, last_page + 1):
        if comentarios_extraidos >= max_comentarios:
            break

        # Monta a URL para a página atual
        url_pagina = url + f"?page={i}"
        response = requests.get(url_pagina)
        content = response.content
        soup = BeautifulSoup(content, 'html.parser')

        # Encontra todos os cards de comentários na página
        cards = soup.find_all(class_='review-card')

        # Processa cada card de comentário individualmente
        for card in cards:
            if comentarios_extraidos >= max_comentarios:
                break

            try:
                # Inicializa variáveis para armazenar informações opcionais
                criticas = None
                seguidores = None

                # Extrai o nome do usuário que fez a crítica
                titulo = card.find('div', class_='meta-title')
                nome = titulo.find('span').text.strip()

                # Busca elementos que contêm informações sobre o perfil do usuário
                items_profils = card.find_all(class_='item-profil')
                if len(items_profils) > 0:
                    # Variável para controlar se já encontramos as informações de seguidores
                    encontrou_seguidores = False
                    
                    # Verificamos cada item para identificar corretamente seguidores e críticas
                    for item in items_profils:
                        if item is not None and item.text.strip() != '':
                            texto_item = item.text.strip()
                            
                            # Verifica se é informação de seguidores
                            if 'seguidor' in texto_item.lower():
                                encontrou_seguidores = True
                                # Limpa e converte o número de seguidores para inteiro
                                try:
                                    seguidores = texto_item.split()[0].replace('.', '')
                                    seguidores = int(seguidores)
                                except (ValueError, IndexError):
                                    seguidores = None
                            
                            # Verifica se é informação de críticas
                            elif 'crítica' in texto_item.lower():
                                # Limpa e converte o número de críticas para inteiro
                                try:
                                    criticas = texto_item.split()[0].replace('.', '')
                                    criticas = int(criticas)
                                except (ValueError, IndexError):
                                    criticas = None

                # Extrai a nota atribuída pelo usuário ao filme
                nota_elem = card.find(class_='stareval-note')
                if nota_elem:
                    nota = nota_elem.text.strip().replace(',', '.')
                    nota = float(nota)  # Converte a nota para float
                else:
                    nota = 0.0

                # Extrai o conteúdo textual da crítica
                conteudo_elem = card.find(class_='review-card-content')
                if conteudo_elem:
                    conteudo = conteudo_elem.text.strip()
                else:
                    conteudo = ""

                # Cria um objeto Comentario e adiciona à lista
                comentario = Comentario(autor=nome, nota=nota, conteudo=conteudo, criticas=criticas, seguidores=seguidores)
                comentarios.append(comentario)
                comentarios_extraidos += 1

            except Exception as e:
                print(f"Erro ao processar comentário: {e}")
                continue

    return comentarios

# Processar cada filme para extrair seus comentários
print(f"Extraindo comentários para {len(MOVIES_DETAILS)} filmes...")

for i, movie in enumerate(MOVIES_DETAILS):
    # Extrair o ID do filme a partir da URL na página de críticas
    print(f"Processando filme {i+1}/{len(MOVIES_DETAILS)}: {movie.name}")

    # Tentamos encontrar o filme na página inicial para obter a URL correta
    response = requests.get(URL + "1")
    content = response.content
    soup = BeautifulSoup(content, 'html.parser')

    # Procuramos por links que contenham o nome do filme
    links = soup.find_all('a', class_='meta-title-link')
    filme_url = None

    # Tentamos encontrar o filme pelo nome
    for link in links:
        if movie.name.lower() in link.text.lower():
            filme_url = 'https://www.adorocinema.com' + link.get('href')
            break

    # Se não encontramos o filme, usamos uma busca
    if filme_url is None:
        nome_busca = movie.name.replace(' ', '+')
        search_url = f"https://www.adorocinema.com/pesquisar/?q={nome_busca}"
        response = requests.get(search_url)
        content = response.content
        soup = BeautifulSoup(content, 'html.parser')

        # Procuramos pelo primeiro resultado que seja um filme
        results = soup.find_all('a', class_='meta-title-link')
        if results:
            filme_url = 'https://www.adorocinema.com' + results[0].get('href')

    # Se encontramos a URL do filme, extraímos os comentários
    if filme_url:
        print(f"URL do filme: {filme_url}")
        movie.comentarios = extrair_comentarios(filme_url)
        print(f"Extraídos {len(movie.comentarios)} comentários")
    else:
        print(f"Não foi possível encontrar a URL para o filme: {movie.name}")

print("Extração de comentários concluída!")

Extraindo comentários para 30 filmes...
Processando filme 1/30: Medianeras: Buenos Aires na Era do Amor Virtual
URL do filme: https://www.adorocinema.com/filmes/filme-112440/
Extraídos 18 comentários
Processando filme 2/30: Wall-E
URL do filme: https://www.adorocinema.com/filmes/filme-123734/
Extraídos 40 comentários
Processando filme 3/30: Hotel Transilvânia
URL do filme: https://www.adorocinema.com/filmes/filme-126079/
Extraídos 40 comentários
Processando filme 4/30: A Era do Gelo 3
URL do filme: https://www.adorocinema.com/filmes/filme-126112/
Extraídos 40 comentários
Processando filme 5/30: Megamente
URL do filme: https://www.adorocinema.com/filmes/filme-127741/
Extraídos 40 comentários
Processando filme 6/30: O Orfanato
URL do filme: https://www.adorocinema.com/filmes/filme-128364/
Extraídos 40 comentários
Processando filme 7/30: Zona Verde
URL do filme: https://www.adorocinema.com/filmes/filme-129054/
Extraídos 20 comentários
Processando filme 8/30: Ilha do Medo
URL do filme: htt

## Criando CSV

In [54]:
def salvar_filmes_csv(filmes, caminho_arquivo_filmes='filmes.csv', caminho_arquivo_comentarios='filmes_comentarios.csv'):

    # Cria diretório para os CSVs se não existir
    diretorio_filmes = os.path.dirname(caminho_arquivo_filmes)
    diretorio_comentarios = os.path.dirname(caminho_arquivo_comentarios)

    if diretorio_filmes and not os.path.exists(diretorio_filmes):
        os.makedirs(diretorio_filmes)

    if diretorio_comentarios and not os.path.exists(diretorio_comentarios):
        os.makedirs(diretorio_comentarios)

    # Lista para armazenar dados dos filmes
    dados_filmes = []

    # Lista para armazenar dados dos comentários
    dados_comentarios = []

    # Para cada filme, adiciona seus dados no CSV de filmes
    for filme in filmes:
        # Adiciona as informações básicas do filme
        linha_filme = {
            'nome': filme.name,
            'nota': filme.grade,
            'resumo': filme.resume
        }
        dados_filmes.append(linha_filme)

        # Se o filme tem comentários, adiciona cada um no CSV de comentários
        if filme.comentarios and len(filme.comentarios) > 0:
            for comentario in filme.comentarios:
                linha_comentario = {
                    'nome_filme': filme.name,
                    'autor_comentario': comentario.autor,
                    'nota_comentario': comentario.nota,
                    'conteudo_comentario': comentario.conteudo,
                    'criticas_autor': comentario.criticas,
                    'seguidores_autor': comentario.seguidores
                }
                dados_comentarios.append(linha_comentario)

    # Cria DataFrames com os dados
    df_filmes = pd.DataFrame(dados_filmes)
    df_comentarios = pd.DataFrame(dados_comentarios)

    # Salva os DataFrames em arquivos CSV
    df_filmes.to_csv(caminho_arquivo_filmes, index=False, encoding='utf-8-sig')
    df_comentarios.to_csv(caminho_arquivo_comentarios, index=False, encoding='utf-8-sig')

    print(f"Dados dos filmes salvos com sucesso em {caminho_arquivo_filmes}")
    print(f"Dados dos comentários salvos com sucesso em {caminho_arquivo_comentarios}")

    return caminho_arquivo_filmes, caminho_arquivo_comentarios

# Salvar os filmes e seus comentários em arquivos CSV separados
caminho_pasta_csv = 'filmes_csv'
if not os.path.exists(caminho_pasta_csv):
    os.makedirs(caminho_pasta_csv)

# Define os caminhos para os arquivos CSV
caminho_arquivo_filmes = os.path.join(caminho_pasta_csv, 'filmes.csv')
caminho_arquivo_comentarios = os.path.join(caminho_pasta_csv, 'filmes_comentarios.csv')

# Salva os dados nos arquivos CSV
arquivos_salvos = salvar_filmes_csv(MOVIES_DETAILS, caminho_arquivo_filmes, caminho_arquivo_comentarios)

# Exibir uma amostra dos dados de filmes salvos
print("\nAmostra dos dados de filmes salvos no CSV:")
df_filmes = pd.read_csv(arquivos_salvos[0])
print(df_filmes.head())

# Exibir uma amostra dos dados de comentários salvos
print("\nAmostra dos dados de comentários salvos no CSV:")
df_comentarios = pd.read_csv(arquivos_salvos[1])
if not df_comentarios.empty:
    print(df_comentarios.head())
else:
    print("Nenhum comentário foi encontrado para os filmes.")


Dados dos filmes salvos com sucesso em filmes_csv\filmes.csv
Dados dos comentários salvos com sucesso em filmes_csv\filmes_comentarios.csv

Amostra dos dados de filmes salvos no CSV:
                                              nome  nota  \
0  Medianeras: Buenos Aires na Era do Amor Virtual   5.0   
1                                           Wall-E   5.0   
2                               Hotel Transilvânia   5.0   
3                                  A Era do Gelo 3   5.0   
4                                        Megamente   5.0   

                                              resumo  
0  Sabe aqueles filmes que começam meio estranho ...  
1  É sempre bom assistir a filmes da era do cinem...  
2  Os filmes de animação têm uma missão complicad...  
3  Quem conhece Manny, Ellie, Sid, Diego, Crash e...  
4  A nova produção da DreamWorks em 3D vem com tu...  

Amostra dos dados de comentários salvos no CSV:
                                        nome_filme          autor_comentario 